In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from bertopic import BERTopic
import pandas as pd 
from transformers.pipelines import pipeline
import os


# Loading articles

In [3]:
df_articles = pd.read_parquet("../files/bbc_news.parquet", engine="pyarrow")
df_articles.head(1)


,title,published_date,authors,description,section,content,link,top_image
0,Saido Berahino: Stoke complete deal to sign We...,2017-01-21,None,Stoke sign West Brom striker Saido Berahino fo...,None,Last updated on .From the section Football\n\n...,http://www.bbc.co.uk/sport/football/38696547,https://ichef.bbci.co.uk/onesport/cps/624/cpsp...


In [4]:
df_articles = df_articles.drop_duplicates('content')

In [5]:
df_articles['text'] = df_articles.apply(lambda x: 'Article Title: ' + x['title'] + '\n' + "Article Body :" + x['content'], axis=1)

In [6]:
docs = df_articles['text'].tolist()

In [7]:
print(docs[0])

Article Title: Saido Berahino: Stoke complete deal to sign West Brom's 23-year-old striker - BBC Sport
Article Body :Last updated on .From the section Football

Stoke have signed West Brom striker Saido Berahino for a fee of £12m on a five-and-a-half-year deal.

The 23-year-old's contract had been due to expire at the end of the season, and the Baggies offered him a new deal for a third time in December.

He has not played since September and his relationship with the club had broken down since the 2014-15 campaign.

"I've had a tough two years but everything happens for a reason. I'm mentally stronger now," Berahino said

"Now I am finally here I just can't wait to start. For Stoke to show their faith in me is unbelievable," he added.

"On match fitness I am not there yet, but I am going to work hard to get myself back so I can help my new team-mates climb the table."

Stoke chief executive Tony Scholes said: "We've signed a young English striker who has already proven his ability in 

# Training Bertopic

In [8]:
topic_model = BERTopic()

In [9]:
topics, probs = topic_model.fit_transform(docs)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

# Pushing model to Hugging Face 


In [10]:
topic_model.push_to_hf_hub(
    "CarlosMorales/bbc_news_topics",
    save_ctfidf=True,
    save_embedding_model="sentence-transformers/all-MiniLM-L6-v2"
)

ctfidf.safetensors:   0%|          | 0.00/52.1M [00:00<?, ?B/s]

topic_embeddings.safetensors:   0%|          | 0.00/2.02M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/CarlosMorales/bbc_news_topics/commit/c84783a40b94a70e3d127c46228d183d9bab9272', commit_message='Add BERTopic model', commit_description='', oid='c84783a40b94a70e3d127c46228d183d9bab9272', pr_url=None, pr_revision=None, pr_num=None)

# Finding articles about conflicts 

In [11]:
topic_model.find_topics("Ukraine war", top_n=5)

([778, 59, 978, 1307, 55],
 [0.66573966, 0.6550449, 0.63287723, 0.62388015, 0.6068135])

In [12]:
similar_topics, similarity = topic_model.find_topics("Israel Palestine War", top_n=5)
topic_model.get_representative_docs(similar_topics[0])

['Article Title: Bowen: Five new realities after four weeks of Israel-Gaza war - BBC News\nArticle Body :The aftermath of a strike on Gaza\'s Jabalia refugee camp earlier this week\n\nOne of the first things to understand about the reportage, analysis and commentary that has poured out since the Hamas attacks of 7 October is that no-one has the full story. Not only is it, as ever, hard to penetrate the fog of war to work out what is happening on the battlefield. The new shape of the conflict between Israelis and Palestinians has not yet emerged.\n\nEvents are still moving fast. Fears that the war could spread are very real. New realities in the Middle East are out there somewhere, but their shape and the way that they will work depend on the way this war goes for the rest of the year, and probably beyond.\n\nHere are a few things that we know, and a few that we do not. The list is not exhaustive. Some people mocked Donald Rumsfeld, US defence secretary at the time of the invasion of Ir